In [177]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 4000
pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_rows', None)

In [178]:
dtypes = {'DOB':'datetime64'}
date_cols = ['DOB','dateReceivedOriginal','dateReceivedCurrent','latestReleaseDate','paroleHearingDate','maxExpirationDateParole','postReleaseMaxExpiration','paroleBoardDischargeDate']
date_cols_unpure = ['earliestRelaseDate','paroleEligibilityDate','conditionalReleaseDate','maxExpirationDate']
transition_table_cols = ['custodyStatus','dateReceivedOriginal','latestReleaseDate','earliestReleaseDate','minSentence','maxSentence','crime1','class1','crime2','class2','crime3','class3','crime4','class4']

In [186]:
df = pd.read_csv("/Users/jpouls/recidiviz/nyrecidiviz/inmates2018.csv",index_col=0,parse_dates=date_cols,na_filter=False)
# df = pd.read_csv("/Users/jpouls/recidiviz/nyrecidiviz/inmates2018.csv",index_col=0,na_filter=False)

# dfs = [pd.read_csv("/Users/jazz/proj/doc-scraper/ny_inmate_data/inmates"+str(year)+".csv",index_col=0,parse_dates=date_cols,na_filter=False) for year in range(2015,2020)]

In [191]:
a1 = df[transition_table_cols]

In [192]:
print("Deleting ", len(a1[a1.earliestReleaseDate == 'NONE'].index), " rows where earliestReleaseDate == 'NONE'")
a1 = a1[a1.earliestReleaseDate != 'NONE']
a1.loc[a1.earliestReleaseDate == 'LIFE', 'earliestReleaseDate'] = '2/22/2222'

Deleting  2  rows where earliestReleaseDate == 'NONE'


In [193]:
a1['end_date'] = a1.latestReleaseDate.combine_first(a1.earliestReleaseDate)

In [199]:
a1['LOS'] = a1.end_date - a1.dateReceivedOriginal
slim = a1[['LOS','crime1','class1','crime2','class2','crime3','class3','crime4','class4']]

In [212]:
slim['crime1Type'] = slim.crime1.str.extract(r'((.*?)\d.*)')[1]
slim.head()

<ipython-input-212-1dfdf367c4df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slim['crime1Type'] = slim.crime1.str.extract(r'((.*?)\d.*)')[1]


,LOS,crime1,class1,crime2,class2,crime3,class3,crime4,class4,crime1Type
DIN,,,,,,,,,,
18A0001,926 days,ATT ASSAULT 2ND,E,ATT CRIM POSS WEAP 2ND,D,BURGLARY 3RD,D,,,ATT ASSAULT
18A0002,338 days,ATT ROBBERY 3RD,E,,,,,,,ATT ROBBERY
18A0003,252 days,CRIM POSS MARIHUANA 3RD,E,CRIM POSS FIREARM,E,,,,,CRIM POSS MARIHUANA
18A0004,804 days,CRIM POSS STOLEN PROP 4TH,E,,,,,,,CRIM POSS STOLEN PROP
18A0005,685 days,ROBBERY 3RD,D,CRIM CONTEMPT 1ST,E,,,,,ROBBERY


In [211]:

crim = [slim.LOS.groupby('crime1Type').quantile(q) for q in range(0.0,1.0,.1)]
crim

TypeError: 'float' object cannot be interpreted as an integer

In [35]:
a1.dtypes

custodyStatus                    object
dateReceivedOriginal     datetime64[ns]
latestReleaseDate        datetime64[ns]
earliestReleaseDate              object
minSentence                     float64
maxSentence                     float64
end_date                 datetime64[ns]
LOS                     timedelta64[ns]
dtype: object